In [1]:
import pandas as pd
from pandas import Series, DataFrame # 最常用的类

In [2]:
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True) # numpy 不要使用科学计数法，小数4位

在很多应用中，**数据可能分布在多个文件或数据库中，抑或以某种不易于分析的格式进行排列**

## 分层索引

**分层索引是`pandas`的重要特性，允许在一个轴向上拥有多个索引层级**

分层索引提供了一种**在更低维度形式中处理更高维度数据的方式**

In [4]:
data = Series(np.random.randn(9), index=[
     list('aaabbccdd'),
    [1,2,3,1,3,1,2,2,3]
])
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [5]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [6]:
type(data.index)

pandas.core.indexes.multi.MultiIndex

通过分层索引对象，也可以称为**部分索引**，允许简洁的选出数据的子集

In [7]:
data['b']

1   -0.555730
3    1.965781
dtype: float64

In [8]:
data[1]

0.47894333805754824

In [10]:
data.loc[: , 1]

a   -0.204708
b   -0.555730
c    1.393406
dtype: float64

In [15]:
data.iloc[1]

0.47894333805754824

In [16]:
data.iloc[5]

1.3934058329729904

In [17]:
data[['b', 'c']]

b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
dtype: float64

分层索引在重塑数据和数据透视图等**分组操作**中扮演了重要角色

In [19]:
data.unstack()

,1,2,3
a,-0.204708,0.478943,-0.519439
b,-0.555730,NaN,1.965781
c,1.393406,0.092908,NaN
d,NaN,0.281746,0.769023


In [22]:
data.unstack().stack()

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [23]:
data1 = data.unstack() 

In [24]:
data1.stack?

In [25]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [26]:
data.index.codes

FrozenList([[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [29]:
data.index.levels

FrozenList([['a', 'b', 'c', 'd'], [1, 2, 3]])

In [30]:
data.index.levels[0]

Index(['a', 'b', 'c', 'd'], dtype='object')

In [32]:
data.index.levels[-1]

Int64Index([1, 2, 3], dtype='int64')

In [34]:
frame = DataFrame(np.arange(12).reshape((4, 3)),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=[['Ohio', 'Ohio', 'Colorado'],
                           ['Green', 'Red', 'Green']]
                  )
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [35]:
frame.index.names = [ 'key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state       Ohio     Colorado
color      Green Red    Green
 key1 key2                   
a     1        0   1        2
      2        3   4        5
b     1        6   7        8
      2        9  10       11

In [36]:
frame.columns

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

In [37]:
frame.index

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           names=[' key1', 'key2'])

In [38]:
frame.index.codes

FrozenList([[0, 0, 1, 1], [0, 1, 0, 1]])

In [39]:
frame.index.levels

FrozenList([['a', 'b'], [1, 2]])

In [47]:
frame.loc['a', 1]

state     color
Ohio      Green    0
          Red      1
Colorado  Green    2
Name: (a, 1), dtype: int64

In [48]:
frame.iloc[:, 1]

 key1  key2
a      1        1
       2        4
b      1        7
       2       10
Name: (Ohio, Red), dtype: int64

In [52]:
frame.loc['a'].iloc[:, :3]

state  Ohio     Colorado
color Green Red    Green
key2                    
1         0   1        2
2         3   4        5

In [50]:
frame['Ohio', 'Red']

 key1  key2
a      1        1
       2        4
b      1        7
       2       10
Name: (Ohio, Red), dtype: int64

### 重排序和层级排序

**重新排列轴上的层级关系**

**按照特定层级的值**对**数据**进行排序

`swaplevel`: 接受**两个层级序号或者层级名称**，返回一个进行了**层级变更**的**新对象**

In [53]:
frame

state       Ohio     Colorado
color      Green Red    Green
 key1 key2                   
a     1        0   1        2
      2        3   4        5
b     1        6   7        8
      2        9  10       11

In [56]:
frame.swaplevel( 0, 1 )

state       Ohio     Colorado
color      Green Red    Green
key2  key1                   
1    a         0   1        2
2    a         3   4        5
1    b         6   7        8
2    b         9  10       11

In [59]:
frame.index.names = [ 'key1', 'key2']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [60]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

`sort_index`**只能在单一层级上对数据进行排序**

在层级变换时，使用`sort_index`以使得结果**按照层级进行字典排序**

In [65]:
frame.swaplevel('key1', 'key2').sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [66]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

### 按层级进行汇总统计

`DataFrame`和`Series`中很多描述性和汇总性统计有一个`level`选项，**通过`level`选项可以指定想要在某个特定的轴上进行聚合**

In [67]:
frame.sum()

state     color
Ohio      Green    18
          Red      22
Colorado  Green    26
dtype: int64

In [68]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [69]:
frame.sum(axis=1)

key1  key2
a     1        3
      2       12
b     1       21
      2       30
dtype: int64

In [70]:
frame.sum(axis=1, level='color')

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 使用DataFrame的列进行索引

通常，**不会使用DataFrame中一个或多个列作为行索引**，反而你**可能要想将行索引移动到DataFrame的列中**

In [74]:
frame = DataFrame(
    {'a': range(7), 'b': range(
        7, 0, -1), 'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'], 'd': [0, 1, 2, 0, 1, 2, 3]}
)
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


`set_index`函数会生成一个新的DataFrame，新的DataFrame**使用多个列作为索引**,默认情况下，这些列从DataFrame中删除，也可以使用`drop=False`保留

In [75]:
frame2 = frame.set_index(['c', 'd'] )
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [76]:
frame.set_index(['c', 'd'] , drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

**`reset_index`是`set_index`的反操作，分层索引的索引层级会被移动到列中**

In [77]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 联合与合并数据集

**包含在pandas对象的数据可以通过多种方式联合在一起**

- `pandas.merge`: 根据**一个或多个键将进行联结**，类似数据库中的连接操作
- `pandas.caoncat`: 使对象在轴向上进行**粘合成堆叠**
- `combine_firsr`方法**允许将堆叠的数据连接在一起**，以**使用一个对象中的值填充另一个对象中的缺失值**

### 数据库风格的DataFrame连接

合并或连接**通过一个或多个键连接行** 来进行连接数据集，这些操作是关系型数据库的核心内容，`pandas`中的`merge`函数主要**将各种 `join`操作算法应用于DataFrame上**

没有指定在哪一行上进行连接，`merge`默认会**自动将重叠列名作为连接的键**，但是，**显示的指定连接键才是好的表现**

In [79]:
df1 = DataFrame({ 'key':list('bbacaab'), 'data1':range(7) })
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [81]:
df2 = DataFrame({'key':list('abd'), 'data2': range(3)})
df2 

,key,data2
0,a,0
1,b,1
2,d,2


In [84]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [85]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [93]:
df3 = DataFrame({'lkey': list('bbacaab'), 'data1': range(7)})
df4 = DataFrame({'rkey': list('abd'), 'data2': range(3)})

In [94]:
#  No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False
pd.merge(df3, df4, left_on='lkey', right_on='rkey') 

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [95]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey', how='left') 

,lkey,data1,rkey,data2
0,b,0,b,1.0
1,b,1,b,1.0
2,a,2,a,0.0
3,c,3,NaN,NaN
4,a,4,a,0.0
5,a,5,a,0.0
6,b,6,b,1.0


In [96]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey', how='right') 

,lkey,data1,rkey,data2
0,a,2.0,a,0
1,a,4.0,a,0
2,a,5.0,a,0
3,b,0.0,b,1
4,b,1.0,b,1
5,b,6.0,b,1
6,NaN,NaN,d,2


In [97]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey', how='outer') 

,lkey,data1,rkey,data2
0,b,0.0,b,1.0
1,b,1.0,b,1.0
2,b,6.0,b,1.0
3,a,2.0,a,0.0
4,a,4.0,a,0.0
5,a,5.0,a,0.0
6,c,3.0,NaN,NaN
7,NaN,NaN,d,2.0


In [98]:
df5 =  DataFrame({'rkey': list('ababd'), 'data1': range(5)})
df5

,rkey,data1
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [103]:
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [104]:
pd.merge(df3, df5, left_on='lkey', right_on='rkey', how='outer')

,lkey,data1_x,rkey,data1_y
0,b,0.0,b,1.0
1,b,0.0,b,3.0
2,b,1.0,b,1.0
3,b,1.0,b,3.0
4,b,6.0,b,1.0
5,b,6.0,b,3.0
6,a,2.0,a,0.0
7,a,2.0,a,2.0
8,a,4.0,a,0.0
9,a,4.0,a,2.0


In [106]:
pd.merge(df3, df5, left_on='lkey', right_on='rkey', how='outer')['lkey'] == 'b' 

0      True
1      True
2      True
3      True
4      True
5      True
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
Name: lkey, dtype: bool

In [107]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

In [108]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [109]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [111]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [112]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [114]:
pd.merge(left, right, on='key1',suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 根据索引合并 Merge On Index

在某些情况下，DataFrame中**用于合并的键是它的索引**，这时需要使用参数`left_index`和`right_index`,表示**索引用来作为合并的键**

In [116]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [117]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [118]:
right1

,group_val
a,3.5
b,7.0


In [120]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [121]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [122]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

In [123]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [124]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [126]:
pd.merge( lefth, righth, left_on=['key1', 'key2'], right_index=True )

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [127]:
pd.merge( lefth, righth, left_on=['key1', 'key2'], right_index=True,how='outer' )

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


![](https://gitee.com/zhao_long/image-store/raw/master/img/20201115151914.png)

In [129]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

In [130]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [131]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [132]:
pd.merge(left2, right2, left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


In [135]:
pd.merge(right2, left1, left_index=True, right_on='key')

,Missouri,Alabama,key,value
1,7.0,8.0,b,1
4,7.0,8.0,b,4
5,9.0,10.0,c,5


In [136]:
pd.merge( left1,right2, left_on='key', right_index=True)

,key,value,Missouri,Alabama
1,b,1,7.0,8.0
4,b,4,7.0,8.0
5,c,5,9.0,10.0


`DataFrame`有一个方便的`join`方法，**用于按照索引合并**，**也可以用于合并多个索引相同或相似但没有重叠列的DataFrame对象**

In [138]:
left2.join(right2)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


In [139]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


由于一些历史原因，DataFrame的`join`方法**进行连接键上的左连接，完全保留左边DataFrame的行索引**， 还**支持在调用DataFrame的某一列上连接传递的DataFrame的索引**

In [140]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [141]:
right1

,group_val
a,3.5
b,7.0


In [142]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


对于**简单的索引-索引合并**， 可以向`join`方法传入一个**`DataFrame`列表**

In [143]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [144]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [145]:
left2.join(right2)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


In [146]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [147]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### 沿轴向连接

另一种数据组合操作的可互换的称为 **拼接，绑定或堆叠**
- `NumPy`的`concatenate`函数
- `pandas`的`concat`函数

In [148]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [149]:
np.concatenate(arr)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [150]:
np.concatenate?

In [151]:
np.concatenate([arr, arr])

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [152]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [153]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [154]:
s1

a    0
b    1
dtype: int64

In [155]:
s2 

c    2
d    3
e    4
dtype: int64

In [156]:
s3

f    5
g    6
dtype: int64

In [159]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [160]:
pd.concat([s1, s2, s3],axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [161]:
pd.concat([s1, s2, s3],axis=1,join='inner')

,0,1,2


In [162]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [163]:
pd.concat([s1, s4])

a    0
b    1
a    0
b    1
f    5
g    6
dtype: int64

In [164]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [167]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [168]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [172]:
pd.concat([s1, s4], axis=1, join='inner', join_axes=[list('abcd')])

TypeError: concat() got an unexpected keyword argument 'join_axes'

In [176]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [177]:
type(result)

pandas.core.series.Series

In [175]:
pd.concat([s1, s1, s3], keys=['one', 'two', 'three'], axis=1)

,one,two,three
a,0.0,0.0,NaN
b,1.0,1.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [178]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [179]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])

In [180]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [181]:
df2

,three,four
a,5,6
c,7,8


In [182]:
pd.concat([df1, df2])

,one,two,three,four
a,0.0,1.0,NaN,NaN
b,2.0,3.0,NaN,NaN
c,4.0,5.0,NaN,NaN
a,NaN,NaN,5.0,6.0
c,NaN,NaN,7.0,8.0


In [183]:
pd.concat([df1, df2], axis=1)

,one,two,three,four
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


In [184]:
pd.concat([df1, df2], keys=['df1', 'df2'])

one  two  three  four
df1 a  0.0  1.0    NaN   NaN
    b  2.0  3.0    NaN   NaN
    c  4.0  5.0    NaN   NaN
df2 a  NaN  NaN    5.0   6.0
    c  NaN  NaN    7.0   8.0

In [185]:
pd.concat([df1, df2], axis=1, keys=['df1', 'df2'])

df1       df2     
  one two three four
a   0   1   5.0  6.0
b   2   3   NaN  NaN
c   4   5   7.0  8.0

In [186]:
pd.concat({'df1':df1, 'df2':df2}, axis=1)

df1       df2     
  one two three four
a   0   1   5.0  6.0
b   2   3   NaN  NaN
c   4   5   7.0  8.0

In [187]:
pd.concat({'df1':df1, 'df2':df2})

one  two  three  four
df1 a  0.0  1.0    NaN   NaN
    b  2.0  3.0    NaN   NaN
    c  4.0  5.0    NaN   NaN
df2 a  NaN  NaN    5.0   6.0
    c  NaN  NaN    7.0   8.0

In [188]:
pd.concat({'df1':df1, 'df2':df2}, axis=1, names=['upper', 'lower'])

upper df1       df2     
lower one two three four
a       0   1   5.0  6.0
b       2   3   NaN  NaN
c       4   5   7.0  8.0

In [189]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [190]:
df1

,a,b,c,d
0,1.246435,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429,-2.001637
2,-0.371843,1.669025,-0.438570,-0.539741


In [191]:
df2

,b,d,a
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614


In [192]:
pd.concat([df1, df2])

,a,b,c,d
0,1.246435,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429,-2.001637
2,-0.371843,1.669025,-0.438570,-0.539741
0,-1.021228,0.476985,NaN,3.248944
1,0.302614,-0.577087,NaN,0.124121


In [193]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,1.246435,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429,-2.001637
2,-0.371843,1.669025,-0.438570,-0.539741
3,-1.021228,0.476985,NaN,3.248944
4,0.302614,-0.577087,NaN,0.124121


In [194]:
pd.concat([df1, df2], axis=1)


,a,b,c,d,b,d,a
0,1.246435,1.007189,-1.296221,0.274992,0.476985,3.248944,-1.021228
1,0.228913,1.352917,0.886429,-2.001637,-0.577087,0.124121,0.302614
2,-0.371843,1.669025,-0.438570,-0.539741,NaN,NaN,NaN


In [195]:
pd.concat([df1, df2], axis=1, ignore_index=True)


,0,1,2,3,4,5,6
0,1.246435,1.007189,-1.296221,0.274992,0.476985,3.248944,-1.021228
1,0.228913,1.352917,0.886429,-2.001637,-0.577087,0.124121,0.302614
2,-0.371843,1.669025,-0.438570,-0.539741,NaN,NaN,NaN


### 联合重叠数据

还有一种数据联合场景，**既不是合并操作，也不是连接操作**，可能有两个数据集，这两个数据集的**索引全部或部分重叠**

In [201]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [202]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [205]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, 5. ])

`combine_first`方法： **对齐逻辑的轴向操作**

In [206]:
a.combine_first(b)

f    0.0
e    2.5
d    2.0
c    3.5
b    4.5
a    5.0
dtype: float64

In [207]:
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=list('abcdef'))
b

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
f    5.0
dtype: float64

In [208]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [209]:
a.combine_first(b)

a    0.0
b    4.5
c    3.5
d    3.0
e    2.5
f    5.0
dtype: float64

In [210]:
a.combine_first?

In [211]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

In [212]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [213]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [214]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 重塑和透视

**重拍列表型数据**有很多基础操作，这些操作被称为 **重塑和透视**

### 使用多层索引进行重塑

多层索引在`DataFrame`中**提供了一种一致性的方式用于重排数据**
- `stack`(堆叠)
    - 会“旋转”或将列中的数据透视到行
- `unstack`(拆堆)
    - 会将行中的数据透视到列

In [215]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                                     name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [218]:
data.index.name, data.columns.name

('state', 'number')

In [220]:
data.stack()

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [221]:
data.stack().unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [222]:
data.stack().unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [223]:
data.stack().unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [225]:
data.stack().unstack('state').unstack('number')

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [226]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

In [227]:
s1

a    0
b    1
c    2
d    3
dtype: int64

In [228]:
s2

c    4
d    5
e    6
dtype: int64

In [233]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [235]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [236]:
data2.unstack().unstack()

a  one    0.0
   two    NaN
b  one    1.0
   two    NaN
c  one    2.0
   two    4.0
d  one    3.0
   two    5.0
e  one    NaN
   two    6.0
dtype: float64

In [237]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [238]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [240]:
result = data.stack()
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [241]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [243]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

In [244]:
df.swaplevel(0,1)

,side,left,right
number,state,,
one,Ohio,0,5
two,Ohio,1,6
three,Ohio,2,7
one,Colorado,3,8
two,Colorado,4,9
three,Colorado,5,10


In [250]:
df.sort_index(level=-1)

side             left  right
state    number             
Colorado one        3      8
         three      5     10
         two        4      9
Ohio     one        0      5
         three      2      7
         two        1      6

In [247]:
df.sort_index()

side             left  right
state    number             
Colorado one        3      8
         three      5     10
         two        4      9
Ohio     one        0      5
         three      2      7
         two        1      6

In [254]:
df.swaplevel(0,1).sort_index()

side             left  right
number state                
one    Colorado     3      8
       Ohio         0      5
three  Colorado     5     10
       Ohio         2      7
two    Colorado     4      9
       Ohio         1      6

### 将“长”透视为“列”

在数据库中CSV中存储**多时间序列的方式**就是所谓的**长格式或者堆叠格式**

长格式：`203 rows × 14 columns`，**行数多**，看起来就是堆叠起来了

In [255]:
data = pd.read_csv('pydata-book/examples/macrodata.csv')
data

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.980,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.150,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.350,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.370,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.540,139.6,3.50,5.2,180.007,2.31,1.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,2008.0,3.0,13324.600,9267.7,1990.693,991.551,9838.3,216.889,1474.7,1.17,6.0,305.270,-3.16,4.33
199,2008.0,4.0,13141.920,9195.3,1857.661,1007.273,9920.4,212.174,1576.5,0.12,6.9,305.952,-8.79,8.91
200,2009.0,1.0,12925.410,9209.2,1558.494,996.287,9926.4,212.671,1592.8,0.22,8.1,306.547,0.94,-0.71
201,2009.0,2.0,12901.504,9189.0,1456.678,1023.528,10077.5,214.469,1653.6,0.18,9.2,307.226,3.37,-3.19


In [256]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [257]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [268]:
data.index = periods.to_timestamp('D')

In [266]:
data

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,13324.600,-3.16,6.0
2008-12-31 23:59:59.999999999,13141.920,-8.79,6.9
2009-03-31 23:59:59.999999999,12925.410,0.94,8.1


In [260]:
data = data.reindex(columns=columns)

In [264]:
data

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2
...,...,...,...
2008-07-01,13324.600,-3.16,6.0
2008-10-01,13141.920,-8.79,6.9
2009-01-01,12925.410,0.94,8.1


In [269]:
data.stack()

date        item   
1959-01-01  realgdp     2710.349
            infl           0.000
            unemp          5.800
1959-04-01  realgdp     2778.801
            infl           2.340
                         ...    
2009-04-01  infl           3.370
            unemp          9.200
2009-07-01  realgdp    12990.341
            infl           3.560
            unemp          9.600
Length: 609, dtype: float64

In [271]:
data.stack().reset_index('item')

,item,0
date,,
1959-01-01,realgdp,2710.349
1959-01-01,infl,0.000
1959-01-01,unemp,5.800
1959-04-01,realgdp,2778.801
1959-04-01,infl,2.340
...,...,...
2009-04-01,infl,3.370
2009-04-01,unemp,9.200
2009-07-01,realgdp,12990.341


In [275]:
data.stack().reset_index().rename( columns = {0:'values'}) 

,date,item,values
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
...,...,...,...
604,2009-04-01,infl,3.370
605,2009-04-01,unemp,9.200
606,2009-07-01,realgdp,12990.341
607,2009-07-01,infl,3.560


In [276]:
ldata =data.stack().reset_index().rename( columns = {0:'values'}) 

In [279]:
ldata.pivot('date', 'item', 'values')

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2
...,...,...,...
2008-07-01,-3.16,13324.600,6.0
2008-10-01,-8.79,13141.920,6.9
2009-01-01,0.94,12925.410,8.1


In [280]:
ldata['value2'] = np.random.randn(len(ldata))
ldata

,date,item,values,value2
0,1959-01-01,realgdp,2710.349,0.523772
1,1959-01-01,infl,0.000,0.000940
2,1959-01-01,unemp,5.800,1.343810
3,1959-04-01,realgdp,2778.801,-0.713544
4,1959-04-01,infl,2.340,-0.831154
...,...,...,...,...
604,2009-04-01,infl,3.370,-0.074155
605,2009-04-01,unemp,9.200,-0.048565
606,2009-07-01,realgdp,12990.341,1.235021
607,2009-07-01,infl,3.560,-0.433295


In [282]:
ldata.pivot('date', 'item')

values                     value2                    
item         infl    realgdp unemp      infl   realgdp     unemp
date                                                            
1959-01-01   0.00   2710.349   5.8  0.000940  0.523772  1.343810
1959-04-01   2.34   2778.801   5.1 -0.831154 -0.713544 -2.370232
1959-07-01   2.74   2775.488   5.3 -0.860757 -1.860761  0.560145
1959-10-01   0.27   2785.204   5.6  0.119827 -1.265934 -1.063512
1960-01-01   2.31   2847.699   5.2 -2.359419  0.332883 -0.199543
...           ...        ...   ...       ...       ...       ...
2008-07-01  -3.16  13324.600   6.0 -0.930342  0.068103  0.845400
2008-10-01  -8.79  13141.920   6.9  0.844963  0.016472  1.850834
2009-01-01   0.94  12925.410   8.1 -1.369179  0.022074  0.887204
2009-04-01   3.37  12901.504   9.2 -0.074155  0.014331 -0.048565
2009-07-01   3.56  12990.341   9.6 -0.433295  1.235021  1.391035

[203 rows x 6 columns]

In [283]:
ldata.set_index(['date', 'item'])

values    value2
date       item                        
1959-01-01 realgdp   2710.349  0.523772
           infl         0.000  0.000940
           unemp        5.800  1.343810
1959-04-01 realgdp   2778.801 -0.713544
           infl         2.340 -0.831154
...                       ...       ...
2009-04-01 infl         3.370 -0.074155
           unemp        9.200 -0.048565
2009-07-01 realgdp  12990.341  1.235021
           infl         3.560 -0.433295
           unemp        9.600  1.391035

[609 rows x 2 columns]

In [285]:
ldata.set_index(['date', 'item']).unstack()

values                     value2                    
item         infl    realgdp unemp      infl   realgdp     unemp
date                                                            
1959-01-01   0.00   2710.349   5.8  0.000940  0.523772  1.343810
1959-04-01   2.34   2778.801   5.1 -0.831154 -0.713544 -2.370232
1959-07-01   2.74   2775.488   5.3 -0.860757 -1.860761  0.560145
1959-10-01   0.27   2785.204   5.6  0.119827 -1.265934 -1.063512
1960-01-01   2.31   2847.699   5.2 -2.359419  0.332883 -0.199543
...           ...        ...   ...       ...       ...       ...
2008-07-01  -3.16  13324.600   6.0 -0.930342  0.068103  0.845400
2008-10-01  -8.79  13141.920   6.9  0.844963  0.016472  1.850834
2009-01-01   0.94  12925.410   8.1 -1.369179  0.022074  0.887204
2009-04-01   3.37  12901.504   9.2 -0.074155  0.014331 -0.048565
2009-07-01   3.56  12990.341   9.6 -0.433295  1.235021  1.391035

[203 rows x 6 columns]

### 将“宽”数据变为“长数据”

`pivot`方法的反操作是`melt`,**与将一列变换为新的DataFrame中的多列不同，它将多列合成一列**，产生一个新的DataFrame，其长度比输入更长

In [286]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df 

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [288]:
pd.melt(df, ['key'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [289]:
pd.melt(df, ['key']).pivot('key', 'variable', 'value')b

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [293]:
pd.melt(df, ['key']).pivot('key', 'variable', 'value').reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [294]:
df.melt?

In [295]:
pd.melt(df,id_vars=['key'], value_vars=['A'] )

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3


In [296]:
pd.melt(df)

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
9,C,7
